In [ ]:
# from transformers import pipeline

# pipe = pipeline("text-to-speech", model="suno/bark-small")
# text = "[clears throat] This is a test ... and I just took a long pause."
# output = pipe(text)

# from IPython.display import Audio
# Audio(output["audio"], rate=output["sampling_rate"])

In [ ]:
# text = "[clears throat] This is a test ... and I just took a long pause."
# output = pipe(text)

# from IPython.display import Audio

# Audio(output["audio"], rate=output["sampling_rate"])

In [3]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf
from IPython.display import Audio

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = ParlerTTSForConditionalGeneration.from_pretrained("parler-tts/parler-tts-tiny-v1").to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts/parler-tts-tiny-v1")

prompt = "Hey, how are you doing today, Buddy!"
description ="Angry voice"
input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
audio_arr = generation.cpu().numpy().squeeze()
sf.write("out.wav", audio_arr, model.config.sampling_rate)
Audio("out.wav")

Config of the text_encoder: <class 'transformers.models.t5.modeling_t5.T5EncoderModel'> is overwritten by shared text_encoder config: T5Config {
  "_name_or_path": "google/flan-t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size

In [ ]:
# import torch
# from IPython.display import Audio
# from parler_tts import ParlerTTSForConditionalGeneration, ParlerTTSStreamer
# from transformers import AutoTokenizer
# from threading import Thread  
# import soundfile as sf

# torch_device = "cuda:0" if torch.cuda.is_available() else "cpu"

# torch_dtype = torch.bfloat16
# model_name = "parler-tts/parler-tts-tiny-v1"

# # need to set padding max length
# max_length = 50

# # load model and tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name) 
# model = ParlerTTSForConditionalGeneration.from_pretrained(
#     model_name,
# ).to(torch_device, dtype=torch_dtype)

# sampling_rate = model.audio_encoder.config.sampling_rate
# frame_rate = model.audio_encoder.config.frame_rate

# def generate(text, description, play_steps_in_s=0.5):
#   play_steps = int(frame_rate * play_steps_in_s)
#   streamer = ParlerTTSStreamer(model, device=torch_device, play_steps=play_steps)
#   # tokenization
#   inputs = tokenizer(description, return_tensors="pt").to(torch_device)
#   prompt = tokenizer(text, return_tensors="pt").to(torch_device)
#   # create generation kwargs
#   generation_kwargs = dict(
#     input_ids=inputs.input_ids,
#     prompt_input_ids=prompt.input_ids,
#     attention_mask=inputs.attention_mask,
#     prompt_attention_mask=prompt.attention_mask,
#     streamer=streamer,
#     do_sample=True,
#     temperature=1.0,
#     min_new_tokens=10,
#   )
#   # initialize Thread
#   thread = Thread(target=model.generate, kwargs=generation_kwargs)
#   thread.start()
#   # iterate over chunks of audio
#   for new_audio in streamer:
#     if new_audio.shape[0] == 0:
#       break
#     print(f"Sample of length: {round(new_audio.shape[0] / sampling_rate, 4)} seconds")
#     yield sampling_rate, new_audio


# # now you can do
# text = "This is a test of the streamer class"
# description = "Jon's talking really fast."

# chunk_size_in_s = 0.5

# for (sampling_rate, audio_chunk) in generate(text, description, chunk_size_in_s):
#     print(audio_chunk.shape)
#     sf.write("tiny_out.wav", audio_chunk, model.config.sampling_rate)
#     Audio("tiny_out.wav")

In [ ]:
import torch
from IPython.display import Audio
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf

torch_device = "cuda:0" if torch.cuda.is_available() else "cpu"

torch_dtype = torch.bfloat16
model_name = "parler-tts/parler-tts-tiny-v1"

# need to set padding max length
max_length = 50

# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = ParlerTTSForConditionalGeneration.from_pretrained(
    model_name,
).to(torch_device, dtype=torch_dtype)

sampling_rate = model.audio_encoder.config.sampling_rate
frame_rate = model.audio_encoder.config.frame_rate


def generate(text, description):
    # tokenization
    inputs = tokenizer(description, return_tensors="pt").to(torch_device)
    prompt = tokenizer(text, return_tensors="pt").to(torch_device)
    # create generation kwargs
    generation_kwargs = dict(
        input_ids=inputs.input_ids,
        prompt_input_ids=prompt.input_ids,
        attention_mask=inputs.attention_mask,
        prompt_attention_mask=prompt.attention_mask,
        do_sample=True,
        temperature=1.0,
        min_new_tokens=10,
    )
    audio = model.generate(**generation_kwargs)
    return audio


# now you can do
text = "This is a test of the streamer class"
description = "Jon's talking really fast."
audio= generate(text, description)
sf.write("tiny_out.wav", audio.cpu().numpy().squeeze(), sampling_rate)
Audio("tiny_out.wav")

In [ ]:
# from IPython.display import Audio
# prompt = "Why so serious?"
# description = "Angry deep voice"

# input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
# prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

# generation = model.generate(input_ids=input_ids, prompt_input_ids=prompt_input_ids)
# audio_arr = generation.cpu().numpy().squeeze()
# sf.write("parler_tts_out.wav", audio_arr, model.config.sampling_rate)
# Audio("parler_tts_out.wav",)